# Training a model with our Own Satellite Image dataset

## Importing Libraries

In [1]:
import os
import glob
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
import cv2
!pip install imagehash
import imagehash

import torchvision.models.segmentation
import torch
import torchdata
import torchvision.transforms as tf

from sklearn.model_selection import train_test_split
from PIL import Image

from pickle import dump

# Check if GPU parallel computing is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 5.9 MB/s eta 0:00:00
cuda


In [2]:
%pip install geedim -q
%pip install torchgeo -q
%pip install rioxarray -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 690.3/690.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.2/535.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
import rioxarray

## Mounting Google Drive

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pathlib import Path
from torchgeo.datasets import RasterDataset, unbind_samples, stack_samples
from torchgeo.samplers import RandomBatchGeoSampler, RandomGeoSampler, Units
from torch.utils.data import DataLoader

In [6]:
# Get the current working directory
current_dir = os.getcwd()

# Print the current working directory
current_dir

'/content'

## Accomodate Paths

In [7]:
# Get the path to the directory
initial_path = "/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data"
dataset_path = "/binary_classification_split/"

# List the contents of the directory
contents = os.listdir(initial_path + dataset_path)

# Print the contents of the directory
for item in contents:
  print(item)

valid_masks
test_landcovers
test_masks
train_masks
train_landcovers
valid_landcovers


## Check Items

In [8]:
test_landcovers_path = initial_path + dataset_path + "/test_landcovers/"

# Contents of the directory
test_landcovers_contents = os.listdir(test_landcovers_path)

print("Test Landcovers items:")
print(len(test_landcovers_contents))

Test Landcovers items:
49


In [9]:
test_masks_path = initial_path + dataset_path + "/test_masks/"

# Contents of the directory
test_masks_contents = os.listdir(test_masks_path)

print("Test Masks items:")
print(len(test_masks_contents))

Test Masks items:
49


In [10]:
train_landcovers_path = initial_path + dataset_path + "/train_landcovers/"

# Contents of the directory
train_landcovers_contents = os.listdir(train_landcovers_path)

print("Train Landcovers items:")
print(len(train_landcovers_contents))

Train Landcovers items:
161


In [11]:
train_masks_path = initial_path + dataset_path + "/train_masks/"

# Contents of the directory
train_masks_contents = os.listdir(train_masks_path)

print("Train Masks items:")
print(len(train_masks_contents))

Train Masks items:
161


In [12]:
valid_landcovers_path = initial_path + dataset_path + "/valid_landcovers/"

# Contents of the directory
valid_landcovers_contents = os.listdir(valid_landcovers_path)

print("Valid Landcovers items:")
print(len(valid_landcovers_contents))

Valid Landcovers items:
21


In [13]:
valid_masks_path = initial_path + dataset_path + "/valid_masks/"

# Contents of the directory
valid_masks_contents = os.listdir(valid_masks_path)

print("Valid Masks items:")
print(len(valid_masks_contents))

Valid Masks items:
21


## This scale function is not working properly (not sure why)

Inspired by:
https://www.geocorner.net/post/artificial-intelligence-for-geospatial-analysis-with-pytorch-s-torchgeo-part-1

In [14]:
# THIS IS NOT WORKING
# We are passing a transform function to the images dataset to scale correctly the values to reflectance values (division by 10,000).
# It only applies to landcovers (not Masks)
# def scale(item: dict):
#     item['image'] = item['image'] / 10000
#     return item

## Creating RasterDatasets for Satellite Images

Inspired by: https://www.geocorner.net/post/artificial-intelligence-for-geospatial-analysis-with-pytorch-s-torchgeo-part-1

In [15]:
from pathlib import Path
root = Path(test_landcovers_path)
assert root.exists()
print(root.as_posix())
test_landcovers = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)

root = Path(test_masks_path)
assert root.exists()
print(root.as_posix())
test_mask = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)
test_mask.is_image = False

from pathlib import Path
root = Path(train_landcovers_path)
assert root.exists()
print(root.as_posix())
train_landcovers = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)

root = Path(train_masks_path)
assert root.exists()
print(root.as_posix())
train_mask = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)
train_mask.is_image = False

from pathlib import Path
root = Path(valid_landcovers_path)
assert root.exists()
print(root.as_posix())
valid_landcovers = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)

root = Path(valid_masks_path)
assert root.exists()
print(root.as_posix())
valid_mask = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)
valid_mask.is_image = False

/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/test_landcovers
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/test_masks
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/train_landcovers
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/train_masks
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/valid_landcovers
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Pr

## Combining Landcovers with Masks by using torchgeo power

In [16]:
train_dset = train_landcovers & train_mask
valid_dset = valid_landcovers & valid_mask
test_dset = test_landcovers & test_mask

## Defining the Hyper-parameters

In [39]:
# Hyperparameters
hparams = {
    'batch_size': 8,
    'num_epochs': 60,
    'num_classes': 2,
    'test_batch_size': 8,
    'learning_rate': 1e-6,
    'weight_decay': 1e-3,
    'log_interval': 100,
}

## Generating samplers

### RandomGeoSampler

https://torchgeo.readthedocs.io/en/latest/api/samplers.html#torchgeo.samplers.RandomGeoSampler.__init__

Parameters:


*   dataset (GeoDataset) – dataset to index from
*   size (Union[tuple[float, float], float]) – dimensions of each patch
*   length (Optional[int]) – number of random samples to draw per epoch (defaults to approximately the maximal number of non-overlapping chips of size size that could be sampled from the dataset)
*   roi (Optional[BoundingBox]) – region of interest to sample from (minx, maxx, miny, maxy, mint, maxt) (defaults to the bounds of dataset.index)
*   units (Units) – defines if size is in pixel or CRS units


In [18]:
train_sampler = RandomGeoSampler(train_landcovers, size=512, length=160, units=Units.PIXELS)
valid_sampler = RandomGeoSampler(valid_landcovers, size=512, length=20, units=Units.PIXELS)
test_sampler = RandomGeoSampler(test_landcovers, size=512, length=50, units=Units.PIXELS)

## Dataloaders attempt

In [19]:
train_dataloader = DataLoader(train_dset, sampler=train_sampler, batch_size=hparams['batch_size'], collate_fn=stack_samples)
valid_dataloader = DataLoader(valid_dset, sampler=valid_sampler, batch_size=hparams['batch_size'], collate_fn=stack_samples)
test_dataloader = DataLoader(test_dset, sampler=test_sampler, batch_size=hparams['batch_size'], collate_fn=stack_samples)

train_batch = next(iter(train_dataloader))
valid_batch = next(iter(valid_dataloader))
test_batch = next(iter(test_dataloader))
train_batch.keys(), valid_batch.keys()

(dict_keys(['crs', 'bbox', 'image', 'mask']),
 dict_keys(['crs', 'bbox', 'image', 'mask']))

## Batch Visualization

The function plot_batch will check automatically the number of items in the batch and if there are masks associated to arrange the output grid accordingly.

In [20]:
from typing import Iterable, List


def plot_imgs(images: Iterable, axs: Iterable, chnls: List[int] = [2, 1, 0], bright: float = 3.):
  for img, ax in zip(images, axs):
    image = img
    print(image.shape)
    # print(image)
    # Replace the tensor with your actual geotiff tensor
    geotiff_tensor = image.permute(1, 2, 0)

    # Check the shape of the tensor
    height, width, bands = geotiff_tensor.shape

    # Assuming it's an RGB image
    if bands == 3:
      # Normalize pixel values to the range [0, 1]
      normalized_tensor = geotiff_tensor / 255.0

      # Reshape the tensor to (height, width, bands) for plotting
      reshaped_tensor = normalized_tensor.reshape((height, width, bands))

      # Display the image using matplotlib
      ax.imshow(reshaped_tensor)
      ax.axis('off')  # Turn off axis labels

    else:
      print("The tensor does not represent an RGB image. Visualization not supported.")


def plot_msks(masks: Iterable, axs: Iterable):
    for mask, ax in zip(masks, axs):
        print(mask.shape)
        # ax.imshow(mask.squeeze().numpy(), cmap='Blues')
        ax.imshow(mask.squeeze().numpy())
        ax.axis('off')


def plot_batch(batch: dict, bright: float = 3., cols: int = 4, width: int = 5, chnls: List[int] = [2, 1, 0]):

    # Get the samples and the number of items in the batch
    samples = unbind_samples(batch.copy())

    # if batch contains images and masks, the number of images will be doubled
    n = 2 * len(samples) if ('image' in batch) and ('mask' in batch) else len(samples)

    # calculate the number of rows in the grid
    rows = n//cols + (1 if n%cols != 0 else 0)

    # create a grid
    _, axs = plt.subplots(rows, cols, figsize=(cols*width, rows*width))

    if ('image' in batch) and ('mask' in batch):
        # plot the images on the even axis
        plot_imgs(images=map(lambda x: x['image'], samples), axs=axs.reshape(-1)[::2], chnls=chnls, bright=bright) #type: ignore

        # plot the masks on the odd axis
        plot_msks(masks=map(lambda x: x['mask'], samples), axs=axs.reshape(-1)[1::2]) #type: ignore

    else:

        if 'image' in batch:
            plot_imgs(images=map(lambda x: x['image'], samples), axs=axs.reshape(-1), chnls=chnls, bright=bright) #type: ignore

        elif 'mask' in batch:
            plot_msks(masks=map(lambda x: x['mask'], samples), axs=axs.reshape(-1)) #type: ignore

In [21]:
plot_batch(train_batch)

Output hidden; open in https://colab.research.google.com to view.

## Data Normalization (Standardization)

Normally, machine learning methods (deep learning included) benefit from feature scaling. That means standard deviation around 1 and zero mean, by applying the following formula (actually normalization is different from standardization, but I will leave the explanation to the reader: https://www.naukri.com/learning/articles/normalization-and-standardization/)

To do that, we need to first find the mean and standard deviation for each one of the 6s channels in the dataset.

Let’s define a function calculate these statistics and write its results in the variables mean and std. We will use our previously installed rasterio package to open the images and perform a simple average over the statistics for each batch/channel. For the standard deviation, this method is an approximation. For a more precise calculation, please refer to: http://notmatthancock.github.io/2017/03/23/simple-batch-stat-updates.htm.

## MyNormalize

https://www.geocorner.net/post/artificial-intelligence-for-geospatial-analysis-with-pytorch-s-torchgeo-part-2

Here we have 6 values in each list. Now we have to use these values to normalize the values every time a batch is created by the dataloader and passed to the trainer. Additionally, if we want to visualize this batch we need to “revert” the standardization, otherwise the true color will not be correct. We will then create a class that will do the trick. We are going to inherit it from the torch.nn.Module class, and define the forward method and also the revert method to “undo” the normalization.


In [22]:
class MyNormalize(torch.nn.Module):
    def __init__(self, mean: List[float], stdev: List[float]):
        super().__init__()

        self.mean = torch.Tensor(mean)[:, None, None]
        self.std = torch.Tensor(stdev)[:, None, None]

    def forward(self, inputs: dict):

        x = inputs["image"][..., : len(self.mean), :, :]

        # if batch
        if inputs["image"].ndim == 4:
            x = (x - self.mean[None, ...]) / self.std[None, ...]

        else:
            x = (x - self.mean) / self.std

        inputs["image"][..., : len(self.mean), :, :] = x

        return inputs

    def revert(self, inputs: dict):
        """
        De-normalize the batch.
        Args:
            inputs (dict): Dictionary with the 'image' key
        """

        x = inputs["image"][..., : len(self.mean), :, :]

        # if batch
        if x.ndim == 4:
            x = inputs["image"][:, : len(self.mean), ...]
            x = x * self.std[None, ...] + self.mean[None, ...]
        else:
            x = x * self.std + self.mean

        inputs["image"][..., : len(self.mean), :, :] = x

        return inputs

## MyNormalize - instantiated

Once the class is defined, we can instantiate it with the mean and std values obtained from our dataset and test the forward pass and the revert pass (code output has been suppressed).

In [23]:
# THIS IS NOT WORKING AS EXPECTED NEITHER
# By using train_batch which has larger length
# normalize = MyNormalize(mean=mean, stdev=std)
# norm_batch = normalize(train_batch)
# plot_batch(norm_batch)

# batch = normalize.revert(norm_batch)
# plot_batch(batch)

# Torchgeo Models

https://torchgeo.readthedocs.io/en/stable/api/models.html#torchgeo.models.resnet18

In [24]:
!pip cache purge

Files removed: 64


In [25]:
!pip install torchgeo lightly

In [26]:
import tempfile

accelerator = "gpu" if torch.cuda.is_available() else "cpu"
default_root_dir = os.path.join(tempfile.gettempdir(), "experiments")

In [27]:
import torch.nn as nn

In [28]:
from typing import Iterable, List, Callable, Optional


def train_loop(
    epochs: int,
    train_dl: DataLoader,
    val_dl: Optional[DataLoader],
    model: nn.Module,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    acc_fns: Optional[List]=None,
    batch_tfms: Optional[Callable]=None
) -> torch.nn.Module:
    # size = len(dataloader.dataset)
    cuda_model = model.cuda()

    for epoch in range(epochs):
        accum_loss = 0
        for batch in train_dl:

            if batch_tfms is not None:
                batch = batch_tfms(batch)

            X = batch['image'].cuda()
            y = batch['mask'].type(torch.long).cuda()
            pred = cuda_model(X)
            loss = loss_fn(pred, y)

            # BackProp
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update the accum loss
            accum_loss += float(loss) / len(train_dl)

        # Testing against the validation dataset
        if acc_fns is not None and val_dl is not None:
            # reset the accuracies metrics
            acc = [0.] * len(acc_fns)

            with torch.no_grad():
                for batch in val_dl:

                    if batch_tfms is not None:
                        batch = batch_tfms(batch)

                    X = batch['image'].type(torch.float32).cuda()
                    y = batch['mask'].type(torch.long).cuda()

                    pred = cuda_model(X)

                    for i, acc_fn in enumerate(acc_fns):
                        acc[i] = float(acc[i] + acc_fn(pred, y)/len(val_dl))

            # at the end of the epoch, print the errors, etc.
            print(f'Epoch {epoch}: Train Loss={accum_loss:.5f} - Accs={[round(a, 3) for a in acc]}')
        else:

            print(f'Epoch {epoch}: Train Loss={accum_loss:.5f}')

    return cuda_model

In [29]:
from sklearn.metrics import jaccard_score

def oa(pred, y):
    flat_y = y.squeeze()
    flat_pred = pred.argmax(dim=1)
    acc = torch.count_nonzero(flat_y == flat_pred) / torch.numel(flat_y)
    return acc

def iou(pred, y):
    flat_y = y.cpu().numpy().squeeze()
    flat_pred = pred.argmax(dim=1).detach().cpu().numpy()
    return jaccard_score(flat_y.reshape(-1), flat_pred.reshape(-1), zero_division=1.)

def loss(p, t):
    return torch.nn.functional.cross_entropy(p, t.squeeze())


## Exploring how FarSeg perfoms

Model documentation: https://torchgeo.readthedocs.io/en/v0.4.0/api/models.html#farseg

Structure:
https://torchgeo.readthedocs.io/en/latest/_modules/torchgeo/models/farseg.html#FarSeg.__init__


In [40]:
from torchgeo.models import FarSeg

# Not sure how to pass below weights to FarSeg model
# from torchgeo.models import ResNet50_Weights
# weights_sent2 = ResNet50_Weights.SENTINEL2_ALL_MOCO

In [41]:
farseg_model_v5 = FarSeg(backbone="resnet50", classes=hparams['num_classes'], backbone_pretrained=False)

In [42]:
import torch.nn as nn

backbone = farseg_model_v5.get_submodule('backbone')

conv = nn.modules.conv.Conv2d(
    in_channels=3,
    out_channels=64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
backbone.register_module('conv1', conv)

pred = farseg_model_v5(torch.randn(3, 3, 512, 512))
pred.shape

torch.Size([3, 2, 512, 512])

In [43]:
# Model path
model_path = "/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/model/"

In [44]:
# List the contents of the directory
model_contents = os.listdir(model_path)

# Print the contents of the directory
for item in model_contents:
  print(item)

my_farseg_model_v1.pt
my_farseg_model_v2.pt
my_farseg_model_v3.pt
my_farseg_model_v4.pt


In [45]:
optimizer = torch.optim.Adam(farseg_model_v5.parameters(), lr=hparams['learning_rate'], weight_decay=hparams['weight_decay'])
my_farseg_model_v5 = train_loop(hparams['num_epochs'], train_dataloader, valid_dataloader, farseg_model_v5, loss, optimizer, acc_fns=[oa, iou])

Epoch 0: Train Loss=0.70500 - Accs=[0.612, 0.538]
Epoch 1: Train Loss=0.70063 - Accs=[0.501, 0.371]
Epoch 2: Train Loss=0.70150 - Accs=[0.525, 0.362]
Epoch 3: Train Loss=0.69755 - Accs=[0.549, 0.318]
Epoch 4: Train Loss=0.69620 - Accs=[0.558, 0.363]
Epoch 5: Train Loss=0.69204 - Accs=[0.526, 0.334]
Epoch 6: Train Loss=0.68748 - Accs=[0.495, 0.248]
Epoch 7: Train Loss=0.69726 - Accs=[0.583, 0.389]
Epoch 8: Train Loss=0.69026 - Accs=[0.583, 0.43]
Epoch 9: Train Loss=0.68249 - Accs=[0.629, 0.377]
Epoch 10: Train Loss=0.68542 - Accs=[0.529, 0.281]
Epoch 11: Train Loss=0.68502 - Accs=[0.607, 0.236]
Epoch 12: Train Loss=0.67769 - Accs=[0.639, 0.324]
Epoch 13: Train Loss=0.67804 - Accs=[0.575, 0.307]
Epoch 14: Train Loss=0.66986 - Accs=[0.656, 0.453]
Epoch 15: Train Loss=0.66949 - Accs=[0.619, 0.286]
Epoch 16: Train Loss=0.67483 - Accs=[0.652, 0.28]
Epoch 17: Train Loss=0.67020 - Accs=[0.645, 0.437]
Epoch 18: Train Loss=0.67181 - Accs=[0.617, 0.178]
Epoch 19: Train Loss=0.67480 - Accs=[0.615,

In [46]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [47]:
# Save the model checkpoint
save_path = model_path + "/my_farseg_model_v5.pt"
print(f"Saving model to {save_path}...")
save_model(my_farseg_model_v5, save_path)
print("Model saved successfully!")

Saving model to /content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/model//my_farseg_model_v5.pt...
Model saved successfully!
